# Intro to `pandas`: the `DataFrame`

We'll explore the Pandas package for simple data handling tasks using geoscience data examples. 

Pandas introduces the concept of a `DataFrame` in Python. If you're familiar with R, it's pretty much the same idea! Useful cheat sheet [here](https://www.datacamp.com/community/blog/pandas-cheat-sheet-python#gs.59HV6BY)

The main purpose of Pandas is to allow easy manipulation of data in tabular form. Perhaps the most important idea that makes Pandas great for data science, is that it will always preserve **alignment** between data and labels.

## Meet the `DataFrame`

The most important data structure in Pandas is `pd.DataFrame`, which is a 2D structure — a table or spreadsheet — that can hold various types of Python objects indexed by a special `index` column (or multiple indices). Columns are nearly always labelled using strings. In a sense, you can think of DataFrames as a dict of names mapped to Series.

An easy way to think about a `DataFrame` is if you imagine it as a table or an Excel spreadsheet, with a lot of superpowers.

Let's define one using a small dataset, defined as a nested `list`:

In [ ]:
data =  [[101, 2.13, 'corroded'],
         [102, 1.45, 'cracked'],
         [104, 4.11, 'pitted'],
         [108, 0.38, 'good'],
         [109, 0.75, 'cracked']]
data

Make a `DataFrame` from `data`

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns=['id', 'roughness', 'condition'])
df

Recall that we earlier instantiated a DataFrame from a dict; we can cast it to a dict as well:

This representation might help explain why the 'first class' index names of the DataFrame are the column names:

As for `Series`, the primary way to index or slice into the DataFrame is with `.loc[]`.

The default `RangeIndex` (positional integer index) can be replaced in the same way as for `Series`:

    df.index = [101, 102, 104, 108, 109]    
    
But often, we want to 'promote' one or more of the DataFrame columns:

## Selecting data

Two useful ways to get at a selection of the data are `.loc[]` (an indexable attribute) and `.query()` (a method).

`.loc` works as for `Series`, but now you can select columns too. Like NumPy indexing, the two 'selectors' are separated by commas:

    df.loc[<row selector>, <column selector>]

For example:

### EXERCISE

- Get the roughness of all cracked samples.
- See if you can get the roughness of all samples that are cracked or pitted.

## Selecting data with `.query()`

We can pass a 'query string' to query, using column names as 'variables' in the string. Eg

Looks like this as a query:

If there's a space in the name, delimit with backticks. But some other characters, eg parentheses, will not work:

## `df.apply()`

Functional programming is a useful paradigm, but it takes a bit of getting used to.

The fundamental concepts are 'map' and 'reduce':

In [ ]:
# Instead of a loop:
def sqrt(x):
    return x**0.5

data = [2, 3, 9]

list(map(sqrt, data))
# There are several ways to do this in Pandas.

In [ ]:
# Aggregate a function over data:
def product(a, b):
    return a * b

from functools import reduce

reduce(product, data)
# This is called 'agg' in Pandas.

Now suppose we have this:

| Surface Roughness     | Description                      |
|-----------------------|----------------------------------|
| 0 µm                  | New (perfectly smooth)           |
| 0.1 µm - 0.5 µm       | Very smooth                      |
| 0.5 µm - 1.6 µm       | Smooth                           |
| 1.6 µm - 3.2 µm       | Moderately rough                 |
| 3.2 µm - 6.3 µm       | Rough                            |
| > 6.3 µm              | Very rough (significant wear)    |

How can we fill a column with these descriptions?

### EXERCISE

Write a function that takes a roughness value and returns an appropriate description.

We cannot pass a Series to this function, but we can pass the function to Pandas via one of the functional methods:

- `map` is defined only on Series. It can use a mapping (dict) or a function.
- `applymap` is defined only on DataFrames.
- `transform` takes a column and produces a column; it will process every column of a DataFrame.
- `agg` reduces a column to a scalar; it will process every column of a DataFrame.
- `apply` smartly tried to `transform` or `agg`, depending on the function.

It's definitely a bit confusing. But in general, I find I tend to use `map` on Series, and `apply` on DataFrames.

For our purpose here, we can use `map`, `transform`, or `apply`.

## Adding data

Add more data (row wise). This is adding a new record; it must have all the columns.

To add a new feature or attribute we want to add a new column — a `Series` or something that can be interpreted as such.

Add a new column with a "complete" list, array or series.

In [ ]:
df['vessel'] = ['FPSO', 'FPSO', 'Handysize', 'Handysize', 'FPSO', 'Unknown']
df

Alternatively, you can broadcast a value or calculation.

In [ ]:
df['source'] = 'Equinor'
df['Ra (µin)'] = df['roughness'] * 39.37  # Convert µm to microinch.
df

### Saving a DataFrame

Before starting the next exercise, let's save the current state:

If anything goes wrong, you can read it again with:

For large datasets you might want to use `pickle`, `feather` or HDF5 formats for better performance. But CSV files are highly portable and human readable, so they are nearly always the best choice for small datasets.

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">


<h3>Exercise</h3>

* Add a new record (row) with a row index of 100 and your own name as the `source`. Make up the other values as needed.
* Add a new boolean Series (with `True` or `False` values) `is_fatigued`. If a record has a `condition` other than `good`, it should be `True`.
* Replace values of 'Unknown' with `np.NaN`.
* Sort the DataFrame by its index.
* Create a subset of the current dataframe with only the `vessel`, `Ra (µin)` and `is_fatigued` columns, in that order; name it `dg`.
</div>

## `groupby`

It's often convenient to gather data according to categorical values, for example "by vessel" in this small dataset.

We can also iterate over the groups:

## Styling

The rich displays in Jupyter can be customized a great deal, for example:

Check out the other examples at https://pandas.pydata.org/docs/user_guide/style.html

## Concatenating DataFrames

Sometimes more data comes along and we need to combine two or more DataFrames. We can use `pd.concat(dfs)` for that. I find I usually want to align the datasets as much as possible, in terms of column names etc, and then do an 'outer' join.

Here's some new, and incomplete, data:

In [ ]:
dh = pd.DataFrame({
 'vessel': {
    200: 'ULCC',
    205: 'ULCC',
    225: 'FPSO',
 },
 'Ra': {
    200: 40,
    205: 65,
    225: 5,
 },
 'is_fatigued': {
    200: False,
    205: True,
    225: False,
 },
})

dh

### EXERCISE

What do we need to do to align these two DataFrames?

1. <u>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</u>
2. <u>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</u>
3. <u>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</u>
4. <u>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</u>
5. <u>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</u>

Do those things!

<div style="background: #e0f0ff; border: solid 2px #d0e0f0; border-radius:3px; padding: 1em; color: navy">

<h3>High performance Pandas</h3>

Note that for very large datasets, there are a few optional dependencies and settings you can use to speed up certain operations (big data, Boolean comparisons, lots of NaNs, etc).

Read more in the docs, eg:

- https://pandas.pydata.org/docs/user_guide/basics.html#accelerated-operations
- https://pandas.pydata.org/docs/getting_started/install.html#performance-dependencies-recommended
</div>

<hr />

<p style="color:gray">©2025 Matt Hall / Equinor. Licensed CC-BY. Remix and share!</p>